# **I. Import library and load data**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from sklearn.model_selection import train_test_split

from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

import lightgbm as lgb


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/machine learning project/processed data.csv")
data = data.drop(columns = ['Unnamed: 0'])
data.head()

,MP,Min,Goals,Assists,CrdY,CrdR,Comp,Shots,Acceleration,Aggression,...,Vision,Volleys,age,name,overall,potential,preferred_foot,position common,market value,wage value
0,34,2983,0.06,0.00,0.15,0.03,Ligue 1,0.54,37,81,...,45,24,33,Yunis Abdelhamid,76,76,Left,defender,3600000.0,23000.0
1,31,2462,0.04,0.00,0.44,0.11,Ligue 1,0.66,50,79,...,65,38,21,Salis Abdul Samed,70,75,Right,midfielder,2200000.0,7000.0
2,34,2956,0.00,0.06,0.27,0.00,Ligue 1,0.91,77,76,...,66,37,28,Laurent Abergel,75,75,Right,midfielder,4900000.0,18000.0
3,24,726,0.00,0.12,0.37,0.00,Bundesliga,2.22,81,39,...,44,53,22,Dickson Abiama,68,76,Right,striker,2700000.0,9000.0
4,30,2536,0.14,0.00,0.07,0.04,Serie A,0.57,64,77,...,65,39,33,Francesco Acerbi,83,83,Left,defender,17500000.0,75000.0


# **II. Convert category data into numeric data and remove columns that don't related in build model**

In [ ]:
# Comp = name of league
data['Comp'].replace(data['Comp'].unique(),[1, 3, 2, 4, 5], inplace=True)
# drop name column and preferred_foot
data = data.drop(columns = ['name','preferred_foot'])

In [ ]:
#convert the categorical variables to dummies
data = pd.get_dummies(data) 
# log tranformation for market vakue
data['market value'] = np.log(data['market value'])
data.head()

,MP,Min,Goals,Assists,CrdY,CrdR,Comp,Shots,Acceleration,Aggression,...,Volleys,age,overall,potential,market value,wage value,position common_defender,position common_goalkeeper,position common_midfielder,position common_striker
0,34,2983,0.06,0.00,0.15,0.03,1,0.54,37,81,...,24,33,76,76,15.096444,23000.0,1,0,0,0
1,31,2462,0.04,0.00,0.44,0.11,1,0.66,50,79,...,38,21,70,75,14.603968,7000.0,0,0,1,0
2,34,2956,0.00,0.06,0.27,0.00,1,0.91,77,76,...,37,28,75,75,15.404746,18000.0,0,0,1,0
3,24,726,0.00,0.12,0.37,0.00,3,2.22,81,39,...,53,22,68,76,14.808762,9000.0,0,0,0,1
4,30,2536,0.14,0.00,0.07,0.04,2,0.57,64,77,...,39,33,83,83,16.677711,75000.0,1,0,0,0


# **III. Split training and test**

In [ ]:
data_build = data.copy()
y = data_build['market value']
x = data_build.drop(columns =['market value'])

x_train, x_test, y_train, y_test = train_test_split(x,y , test_size = 0.33, random_state = 3 )

# **IV.Emsemble models : LightGBM**

## 1.Hyperparameter turing

In [ ]:
regressor = lgb.LGBMRegressor()
params = {
    'n_estimators': [100,200,1000, 2000],
    'learning_rate': [x/100 for x in range(1, 21, 2)],
    'reg_alpha': [0, 1e-1, 1, 2, 5, 7, 10, 50, 100],
    'reg_lambda': [0, 1e-1, 1, 5, 10, 20, 50, 100],
    'max_depth':[int(x) for x in range(3, 11, 2)]
}
scoring_fnc = 'neg_mean_squared_error'
grid = GridSearchCV(estimator=regressor, param_grid=params, scoring= scoring_fnc, cv= 5 , verbose= 5)
grid.fit(x_train, y_train)

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
[CV 1/5] END learning_rate=0.19, max_depth=3, n_estimators=1000, reg_alpha=0.1, reg_lambda=0;, score=-0.015 total time=   0.4s
[CV 2/5] END learning_rate=0.19, max_depth=3, n_estimators=1000, reg_alpha=0.1, reg_lambda=0;, score=-0.011 total time=   0.4s
[CV 3/5] END learning_rate=0.19, max_depth=3, n_estimators=1000, reg_alpha=0.1, reg_lambda=0;, score=-0.012 total time=   0.4s
[CV 4/5] END learning_rate=0.19, max_depth=3, n_estimators=1000, reg_alpha=0.1, reg_lambda=0;, score=-0.018 total time=   0.4s
[CV 5/5] END learning_rate=0.19, max_depth=3, n_estimators=1000, reg_alpha=0.1, reg_lambda=0;, score=-0.015 total time=   0.5s
[CV 1/5] END learning_rate=0.19, max_depth=3, n_estimators=1000, reg_alpha=0.1, reg_lambda=0.1;, score=-0.015 total time=   0.4s
[CV 2/5] END learning_rate=0.19, max_depth=3, n_estimators=1000, reg_alpha=0.1, reg_lambda=0.1;, score=-0.013 total time=   0.4s
[CV 3/5] END learning_rate=0.19, max_depth=3, n_es

GridSearchCV(cv=5, estimator=LGBMRegressor(),
             param_grid={'learning_rate': [0.01, 0.03, 0.05, 0.07, 0.09, 0.11,
                                           0.13, 0.15, 0.17, 0.19],
                         'max_depth': [3, 5, 7, 9],
                         'n_estimators': [100, 200, 1000, 2000],
                         'reg_alpha': [0, 0.1, 1, 2, 5, 7, 10, 50, 100],
                         'reg_lambda': [0, 0.1, 1, 5, 10, 20, 50, 100]},
             scoring='neg_mean_squared_error', verbose=5)

In [ ]:
# get best parameter
grid.best_estimator_.get_params()

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.05,
 'max_depth': 3,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 1000,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.1,
 'reg_lambda': 1,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

# 2. Training model with hyperparameter 

In [ ]:
# Training model with hyperparameter 
model = lgb.LGBMRegressor(n_estimators = 1000, learning_rate = 0.05, reg_alpha = 0.1,
                          reg_lambda  = 1, max_depth = 3)
model.fit(x_train, y_train)
#predict
y_pre = model.predict(x_test) 
print('Root mean squared' ,(mean_squared_error(y_pre,y_test )**(1/2)))
print('R2-scored ', r2_score(y_pre, y_test))

Root mean squared 0.09735391781435403
R2-scored  0.9939875216024947
